In [ ]:
from display.display_figures import display_data, display_REstim, display_REstim_by_dpt

# For data reading purposes
from include.load_data.get_counts import get_real_counts_by_county
from include.load_data import get_counts

# For independent denoising
from include.optim_tools.sliding_median import sliding_median

# For Rt estimation
from include.estim.Rt_MLE import Rt_MLE
from include.estim.Rt_Gamma import Rt_Gamma
from include.estim.Rt_Univariate import Rt_Univariate
from include.estim.Rt_UnivariateOutliers import Rt_Univariate_Outliers
from include.estim.Rt_Multivariate import Rt_Multivariate

## Univariate reproduction number estimations on real-world infection counts
### Data fetching

dataBasis choice between `SPF` (Santé Publique France) and `JHU` (Johns Hopkins University)
&nbsp;
country choice should only be
   * `France` for `SPF`
   * any country available at https://coronavirus.jhu.edu/data for `JHU`

In [ ]:
# Choice of country, dates, regularization parameters & computation
dataBasis = 'JHU'
country = 'France'
firstDay = '2021-11-01'  # first day of the chosen time period
lastDay = '2022-08-03'  # last day of the chosen time period

# Fetch data and associated dates
data, output = get_counts.get_real_counts(country, firstDay, lastDay, dataBasis)
dates = list(output['dates'])
display_data(dates, data, "Daily new cases %s for %s, %s and %s" % (dataBasis, country, firstDay, lastDay));

### Maximum Likelihood Estimator
$$\mathsf{R}^{\mathsf{MLE}} = \frac{ \mathsf{Z}}{\mathsf{Z}\Phi}$$

In [ ]:
# Analytic computation
Rt_MLE, datesMLE, dataMLE = Rt_MLE(dates, data)

# Display figures
display_REstim(datesMLE, dataMLE, Rt_MLE, 'MLE');

### Bayesian Estimator

Following the article "A New Framework and Software to Estimate Time-Varying Reproduction Numbers During Epidemics", Cori et al., 2013, Am. Journal of Epidemiology, R estimation is made using Bayesian framework resulting in:

$$\mathsf{R}_t^\Gamma = \frac{1 + \sum_{s=t-\tau}^{t}\mathsf{Z}_s}{5 + \sum_{s=t-\tau}^{t}(\Phi\star\mathsf{Z})_s}$$

Here $\tau = 15$

In [ ]:
# Analytic computation
Rt_Gamma, datesGamma, dataGamma = Rt_Gamma(dates, data)

# Display results
display_REstim(datesGamma, dataGamma, Rt_Gamma, 'Gamma');

### Penalized Likelihood estimation

1. $\mathsf{Z}^{\mathsf{denoised}} = \mathsf{slidingMAD}(\mathsf{Z}, \alpha)$
    &nbsp;

    with sliding Median Absolute Deviation (MAD) that can be tuned using $\alpha \geq 0$:
    - small $\alpha$ : lot of data denoised
    - great $\alpha$ : few data denoised

2. Solving the Penalized Likelihood functional with Chambolle-Pock algorithm

   $$\mathsf{R}^{\mathsf{U}} \in \mathrm{arg\,min}_{\mathsf{R} \in \mathbb{R}^{\mathsf{T}}} -\ln \mathsf{Likelihood}(\mathsf{Z}^{\mathsf{denoised}}, \mathsf{R}) +\mu_\mathsf{R} \|\boldsymbol{\mathsf{D}}_2\mathsf{R}\|_1$$

In [ ]:
# Choice of denoising parameter alpha and regularization parameter muR
alpha = 0.1
muR = 50

# Denoising with sliding median then solving with Chambolle-Pock algorithm
data_denoised = sliding_median(data, alpha)
Rt_U, datesU, data_denoisedCrop = Rt_Univariate(dates, data_denoised, muR=muR)

# Display results
dataCropped = data[1:]  # data[0] is used for burn-in
Ot_U = dataCropped - data_denoisedCrop
display_REstim(datesU, dataCropped, Rt_U, 'U', OEstimate=Ot_U);

### Joint estimation

Solving the following Univariate variational estimator with temporal regularization with misreported counts O explicit modelisation, using Chambolle-Pock algorithm

$$\mathsf{R}^{\mathsf{U-O}}, \mathsf{O}^{\mathsf{U-O}} \in \mathrm{arg\,min}_{\mathsf{R}, \mathsf{O} \in \mathbb{R}^{\mathsf{T}}} -\ln \mathsf{Likelihood}(\mathsf{Z}, \mathsf{R}, \mathsf{O}) +\lambda_\mathsf{R} \|\boldsymbol{\mathsf{D}}_2\mathsf{R}\|_1 + \lambda_\mathsf{O} \|\mathsf{O}\|_1$$

* A greater $\lambda_R$ means less slope ruptures
* A greater $\lambda_O$ means less denoising

For more examples, see [include/settings.RegularizationSettings](include/settings.py)

In [ ]:
# Choice of denoising parameter alpha and regularization parameter muR
lambdaR = 3.5
lambdaO = 0.03

# Solving with Chambolle-Pock algorithm
Rt_UO, Ot_UO, output = Rt_Univariate_Outliers(dates, data, lambdaR=lambdaR, lambdaO=lambdaO)

# Display results
datesUO = list(output['dates'])
dataUO = list(output['data'])
display_REstim(datesUO, dataUO, Rt_UO, 'U-O', OEstimate=Ot_UO);

In [ ]:
# Gather all methods
methods = ['MLE', 'Gamma', 'U', 'U-O']

REstimations = {'MLE': Rt_MLE,
                'Gamma': Rt_Gamma,
                'U': Rt_U,
                'U-O': Rt_UO}

OEstimations = {'U': dataCropped - data_denoisedCrop,
                'U-O': Ot_UO}

# Display comparison between methods
display_REstim(datesUO, dataUO, REstimations, methods, OEstimate=OEstimations, comparison=True);

## Multivariate reproduction number estimations on real-world infection counts

In [ ]:
# Choice of dates
firstDay = '2021-11-03'
lastDay = '2022-03-04'

# Retrieve associated infection counts by county
data, optionsSpat = get_real_counts_by_county(firstDay, lastDay)

In [ ]:
# Choice of regularization parameters
lambdaR = 50  # temporal regularization
lambdaS = 0.005  # spatial regularization

REstimate, datesUpdated, dataCrop = Rt_Multivariate(data, lambdaR, lambdaS, optionsSpat)

In [ ]:
from scipy.io import loadmat
# savemat('test_demo.mat', {'REstimate': REstimate,
#                           'datesUpdated': datesUpdated,
#                           'dataCrop': dataCrop,
#                           'counties': optionsSpat['counties']})

file = loadmat('test_demo.mat', squeeze_me=True)
REstimate = file['REstimate']
datesUpdated = file['datesUpdated']
dataCrop = file['dataCrop']
counties = file['counties'] #  optionsSpat['counties']

selectedCounties = ['2A', '2B',  '23', '75']
display_REstim_by_dpt(datesUpdated, dataCrop, REstimate, selectedCounties);